In [1]:
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from math import sqrt

In [ ]:
openpose_json_dir = Path("outputs/openpose/2/json")
baseline_json_dir = Path("outputs/simple_landmarks/2/json")
openpose_overlay_dir = Path("outputs/openpose/2/overlays")
frames_dir = Path("frames/frames2")

In [5]:
def draw_keypoints(image, points, color, label_prefix=""):
    for i, (x, y) in enumerate(points):
        if x > 0 and y > 0:
            cv2.circle(image, (int(x), int(y)), 5, color, -1)
            cv2.putText(image, f"{label_prefix}{i+1}", (int(x)+5, int(y)-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1, cv2.LINE_AA)

In [6]:
def euclidean_distance(p1, p2):
    return sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

In [7]:
key_order = [
    "nose", "left_eye", "right_eye", "thumb",
    "pointer_finger", "middle_finger", "ring_finger", "pinky_finger"
]

In [ ]:
openpose_key_indices = {
    "nose": 0,
    "left_eye": 15,
    "right_eye": 16,
    "thumb": 4,
    "pointer_finger": 7,
    "middle_finger": 6,
    "ring_finger": 3,
    "pinky_finger": 2
}

In [ ]:
all_frame_errors = []

for i in range(1, 7):
    frame_name = f"frame_{i:02d}"
    openpose_json_path = openpose_json_dir / f"{frame_name}_keypoints.json"
    baseline_path = baseline_json_dir / f"{frame_name}.json"

    if not openpose_json_path.exists():
        continue
    if not baseline_path.exists():
        continue

    # Load OpenPose data
    with open(openpose_json_path, "r") as f:
        data = json.load(f)
    people = data.get("people", [])
    if not people:
        continue
    keypoints = people[0]["pose_keypoints_2d"]

    openpose_points_all = [(keypoints[j], keypoints[j+1]) for j in range(0, len(keypoints), 3)]

    openpose_points = []
    for k in key_order:
        idx = openpose_key_indices[k]
        if idx < len(openpose_points_all):
            openpose_points.append(openpose_points_all[idx])
        else:
            openpose_points.append((0, 0))

    # Load baseline data
    with open(baseline_path, "r") as f:
        baseline_data = json.load(f)
    baseline_points = [(baseline_data[k]["coordinates"]["x"], baseline_data[k]["coordinates"]["y"]) for k in key_order]

    # Load image
    img_path = openpose_overlay_dir / f"{frame_name}_rendered.png"
    if not img_path.exists():
        img_path = frames_dir / f"{frame_name}.jpg"
    img = cv2.imread(str(img_path))
    if img is None:
        continue
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    draw_keypoints(img, baseline_points, (0, 255, 0), "B")
    draw_keypoints(img, openpose_points, (255, 0, 0), "O")

    # Compute errors
    errors = []
    for b, o in zip(baseline_points, openpose_points):
        dist = euclidean_distance(b, o)
        errors.append(dist)
        cv2.line(img, (int(b[0]), int(b[1])), (int(o[0]), int(o[1])), (255, 255, 0), 1)

    mean_error = np.mean(errors)
    max_error = np.max(errors)
    all_frame_errors.append(mean_error)

    # Display results
    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.title(f"{frame_name}: Mean error={mean_error:.2f}px | Max={max_error:.2f}px")
    plt.axis("off")
    plt.show()

    print(f"\nFrame {i:02d} Results")
    for k, dist in zip(key_order, errors):
        print(f"  {k:15s}: {dist:7.2f}px")
    print(f"  Mean error: {mean_error:.2f}px | Max error: {max_error:.2f}px")

print(f"Average mean error across frames: {np.mean(all_frame_errors):.2f}px")
print(f"Best (lowest) frame mean error: {np.min(all_frame_errors):.2f}px")
print(f"Worst (highest) frame mean error: {np.max(all_frame_errors):.2f}px")